# Email Service - Send to all consumers


## Download dependencies


In [75]:
import sys
!{sys.executable} -m pip install openai bs4 pandas requests smtp_email_sender psycopg2 dotenv


  Using cached openai-2.8.1-py3-none-any.whl.metadata (29 kB)
  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached pandas-2.3.3-cp39-cp39-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached smtp_email_sender-1.0.4-py3-none-any.whl.metadata (7.3 kB)
  Using cached psycopg2-2.9.11-cp39-cp39-macosx_10_9_universal2.whl
  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)
  Using cached anyio-4.12.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.12.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.me

In [76]:
!pip install python-docx

  Using cached python_docx-1.2.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached lxml-6.0.2-cp39-cp39-macosx_10_9_universal2.whl.metadata (3.6 kB)
Using cached python_docx-1.2.0-py3-none-any.whl (252 kB)
Using cached lxml-6.0.2-cp39-cp39-macosx_10_9_universal2.whl (8.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [python-docx] [python-docx]


## Get all environmental variables

In [77]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads the .env file in the current directory

open_ai_key = os.getenv("OPENAI_KEY")
gmail_email = os.getenv("APP_GMAIL_EMAIL")
gmail_app_password = os.getenv("APP_GMAIL_PWD")
postgres_connection_string = os.getenv("POSTGRES_CONNECTION_STRING")

## Prompt engineering

This will be used to guide LLM. Open the txt files


### Zero shot prompt for text classification

In [78]:
with open("political_text_classifier.txt", "r", encoding="utf-8") as file:
    political_text_classifier = file.read()

### Few shot prompt for summarizer 

In [79]:
with open("political_text_summarizer.txt", "r", encoding="utf-8") as file:
    political_text_summarizer = file.read()

## Define AI functions

This uses OpenAI SDK and GPT 4o as base LLM


### Set up OpenAI client


In [80]:
from openai import OpenAI

client = OpenAI(api_key=open_ai_key)

### Political text classification


In [81]:
def classifyText(input_text):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": political_text_classifier
            },
            {"role": "user", "content": input_text}
        ]
    )
    return response.choices[0].message.content.strip().split()[0]

### Summarizer of legislative text


In [82]:
def summarizeText(input_text: str):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": political_text_summarizer},
            {"role": "user", "content": "Summarize the following legislation: " + input_text}
        ]
    )
    return response.choices[0].message.content.strip()

## Build webscraper to get NYC legislation

BeautifulSoup will be used to access HTML of NYC Legistar website. We will save the scraped data in a Pandas dataframe for further processing


### Scrape council committee meetings


In [83]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

request_url = "https://legistar.council.nyc.gov/Calendar.aspx?Mode=This+Week"
web_html = requests.get(request_url).text
soup = BeautifulSoup(web_html, "html.parser")
table = soup.find('table', id='ctl00_ContentPlaceHolder1_gridCalendar_ctl00')

council_meetings = []

# Skip the header row which is index 0
# Only get the 2 most recent meetings
for tr in table.find_all('tr')[1:]:  
    cells = tr.find_all('td')

    committee = cells[0].get_text(strip=True)
    date = cells[1].get_text(strip=True)
    meeting_time = cells[3].get_text(strip=True)
    
    if meeting_time == "Deferred":
        continue
    
    # Get the agenda link (from the 7th column)
    meeting_detail = cells[6].find('a')
    meeting_detail_aspx = meeting_detail['href']    
    
    if len(council_meetings) < 2:
        council_meetings.append({
            'Date': date,
            'Committee': committee,
            'Meeting Details': meeting_detail_aspx
        })
    else:
        break

### View council meetings in a formatted manner


In [84]:
df = pd.DataFrame(council_meetings)
df

,Date,Committee,Meeting Details
0,11/25/2025,City Council Stated Meeting,MeetingDetail.aspx?ID=1359999&GUID=80C2CF9F-9E...
1,11/25/2025,Committee on Women and Gender Equity,MeetingDetail.aspx?ID=1355584&GUID=629C1A78-9F...


## Scrape Meeting Details + Process through AI

- Iterate through each meeting detail and iterate through each "Introduction" legislative text
- Run the text through the 2 AI helper functions


### Define all categories to store legislation summaries

In [85]:
categories = {
    "Immigration": [],
    "Economy": [],
    "Civil": []
}

### Scrap legislation document from meeting details

In [ ]:
from io import BytesIO
from docx import Document

for meeting in council_meetings:
    meeting_details_url = f"https://legistar.council.nyc.gov/{meeting['Meeting Details']}"
    meeting_details_html = requests.get(meeting_details_url).text
    soup = BeautifulSoup(meeting_details_html, "html.parser")  
    table = soup.find('table', id='ctl00_ContentPlaceHolder1_gridMain_ctl00')
    legislation_file = []
    
    for tr in table.find_all('tr')[1:]:  # Skip header row
        cells = tr.find_all('td')
        file_type = cells[6].get_text(strip=True)
        if file_type != "Introduction":
            continue
        
        file_locator = cells[0].find('a')['href']
        if len(legislation_file) > 2:
            break
        
        legislation_file.append(file_locator)
    
    for file_locator in legislation_file:
        response = requests.get(f"https://legistar.council.nyc.gov/{file_locator}").text
        soup = BeautifulSoup(response, "html.parser")

        span_body = soup.find('span', id="ctl00_ContentPlaceHolder1_lblAttachments2")
        legislation_pdf_link = span_body.find_all('a')[2]['href']
        file_number = soup.find('span', id="ctl00_ContentPlaceHolder1_lblFile2").get_text(strip=True)
        status = soup.find('span', id="ctl00_ContentPlaceHolder1_lblStatus2").get_text(strip=True)

        fetched_document = requests.get(f"https://legistar.council.nyc.gov/{legislation_pdf_link}")
        doc = Document(BytesIO(fetched_document.content))    
        legislation_text = '\n'.join([para.text for para in doc.paragraphs])

        category = classifyText(legislation_text)
        summarized_category = summarizeText(legislation_text)

        categories[category].append({
            "Name": file_number,
            "Summarized": summarized_category
        })

## Email to recipients


### Get recipients list
The recipients emails are stored in Neon postgres DB

In [90]:
import pandas as pd

connection = psycopg2.connect(postgres_connection_string)

# Read directly into pandas DataFrame
df = pd.read_sql_query(
    "SELECT email FROM email_subscriptions;",
    connection
)

# Convert to list
recipients = df['email'].tolist()

connection.close()

print(recipients)

['krishivthakuria@gmail.com', 'itskrishiv@gmail.com', 'hemitvpatel@gmail.com']


/var/folders/66/30cf38qx45jfjkzv7xws6vcc0000gn/T/ipykernel_8632/1567177765.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(


### Develop a HTML page

In [ ]:
def parse_summary_to_bullets(summary):
        """Parse AI-generated summary with asterisks into HTML bullet points"""
        lines = [line.strip()[1:].strip() for line in summary.split('\n') 
                 if line.strip().startswith('*')]
        
        if not lines:
            return f'<p>{summary}</p>'
        
        bullets = ''.join([f'<li>{line}</li>' for line in lines])
        return f'<ul>{bullets}</ul>'
    
def render_bills(category_name):
        """Render bills for a specific category"""
        bills = categories.get(category_name, [])
        
        if not bills:
            return '<p style="color: #737373; font-style: italic;">No legislation to report this week.</p>'
        
        bills_html = ''
        for bill in bills:
            summary_html = parse_summary_to_bullets(bill['Summarized'])
            
            bills_html += f'''
                <div class="bill-item">
                    <div class="bill-header">
                        <span class="bill-name">{bill['Name']}</span>
                    </div>
                    <div class="bill-summary">
                        {summary_html}
                    </div>
                </div>
            '''
        
        return bills_html
    
    # Generate the complete HTML
html = f'''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>NYC Legislation Update</title>
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        body {{
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, 'Helvetica Neue', Arial, sans-serif;
            background-color: #fafafa;
            padding: 20px;
            line-height: 1.6;
        }}
        
        .container {{
            max-width: 650px;
            margin: 0 auto;
            background-color: #ffffff;
            border: 1px solid #e0e0e0;
        }}
        
        .header {{
            background-color: #dc2626;
            color: white;
            padding: 40px 30px;
            text-align: center;
        }}
        
        .header h1 {{
            font-size: 32px;
            font-weight: 700;
            margin-bottom: 8px;
        }}
        
        .header p {{
            font-size: 15px;
            font-weight: 400;
        }}
        
        .content {{
            padding: 40px 30px;
        }}
        
        .section {{
            margin-bottom: 50px;
        }}
        
        .section:last-child {{
            margin-bottom: 0;
        }}
        
        .section-title {{
            font-size: 24px;
            font-weight: 700;
            color: #dc2626;
            margin-bottom: 24px;
            padding-bottom: 8px;
            border-bottom: 2px solid #dc2626;
        }}
        
        .bill-item {{
            border-left: 3px solid #dc2626;
            padding: 20px 20px 20px 24px;
            margin-bottom: 24px;
            background-color: #fafafa;
        }}
        
        .bill-header {{
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-bottom: 16px;
        }}
        
        .bill-name {{
            font-weight: 700;
            font-size: 18px;
            color: #1a1a1a;
        }}
        
        .status-badge {{
            padding: 6px 14px;
            font-size: 12px;
            font-weight: 600;
            text-transform: uppercase;
            letter-spacing: 0.5px;
            border: 1px solid;
        }}
        
        .status-active {{
            background-color: #ffffff;
            color: #dc2626;
            border-color: #dc2626;
        }}
        
        .status-pending {{
            background-color: #ffffff;
            color: #525252;
            border-color: #525252;
        }}
        
        .status-passed {{
            background-color: #dc2626;
            color: #ffffff;
            border-color: #dc2626;
        }}
        
        .bill-summary {{
            font-size: 15px;
            color: #404040;
        }}
        
        .bill-summary ul {{
            margin: 0;
            padding-left: 20px;
        }}
        
        .bill-summary li {{
            margin-bottom: 8px;
        }}
        
        .bill-summary li:last-child {{
            margin-bottom: 0;
        }}
        
        .footer {{
            background-color: #fafafa;
            padding: 24px 30px;
            text-align: center;
            font-size: 13px;
            color: #737373;
            border-top: 1px solid #e0e0e0;
        }}
        
        .footer a {{
            color: #dc2626;
            text-decoration: none;
            font-weight: 500;
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>NYC Legislation Update</h1>
            <p>Your weekly digest of important city legislation</p>
        </div>
        
        <div class="content">
            <!-- Immigration Section -->
            <div class="section">
                <h2 class="section-title">Immigration</h2>
                {render_bills('Immigration')}
            </div>
            
            <!-- Civil Section -->
            <div class="section">
                <h2 class="section-title">Civil Rights</h2>
                {render_bills('Civil')}
            </div>
            
            <!-- Economy Section -->
            <div class="section">
                <h2 class="section-title">Economy</h2>
                {render_bills('Economy')}
            </div>
        </div>
        
        <div class="footer">
            <p>© 2024 NYC Legislation Newsletter | <a href="#">Unsubscribe</a></p>
        </div>
    </div>
</body>
</html>'''


### Send emails

In [92]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

SENDER_EMAIL = gmail_email
SENDER_PASSWORD = gmail_app_password  
SUBJECT = "Weekly NYC Legislation Update - Next Voters"

for recipient_email in recipients:
    try:
        msg = MIMEMultipart('alternative')
        msg['Subject'] = SUBJECT
        msg['From'] = SENDER_EMAIL
        msg['To'] = recipient_email

        html_part = MIMEText(html, 'html')
        msg.attach(html_part)

        print(f"Connecting to SMTP server for {recipient_email}...")
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.set_debuglevel(1)
        server.starttls()

        print("Logging in...")
        server.login(SENDER_EMAIL, SENDER_PASSWORD)

        server.send_message(msg)
        print(f"✓ Email sent successfully to {recipient_email}")

        server.quit()

    except smtplib.SMTPAuthenticationError as e:
        print(f"✗ Authentication failed: {e}")
        print("\nTroubleshooting:")
        print("1. Make sure 2-Factor Authentication is enabled on your Google account")
        print("2. Generate an App Password at: https://myaccount.google.com/apppasswords")
        print("3. Use the App Password (16 characters, no spaces) instead of your regular password")

    except Exception as e:
        print(f"✗ Error sending email to {recipient_email}: {e}")

print("\nAll emails processed.")

Connecting to SMTP server for krishivthakuria@gmail.com...


send: 'ehlo 1.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.ip6.arpa\r\n'
reply: b'250-smtp.gmail.com at your service, [99.235.156.91]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [99.235.156.91]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'


Logging in...


send: 'ehlo 1.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.ip6.arpa\r\n'
reply: b'250-smtp.gmail.com at your service, [99.235.156.91]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [99.235.156.91]\nSIZE 35882577\n8BITMIME\nAUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'AUTH PLAIN AG5leHR2b3RlcnNAZ21haWwuY29tAHhycWFncXVzaXNqbXpoamQg\r\n'
reply: b'235 2.7.0 Accepted\r\n'
reply: retcode (235); Msg: b'2.7.0 Accepted'
send: 'mail FROM:<nextvoters@gmail.com> size=24122\r\n'
reply: b'250 2.1.0 OK e9e14a558f8ab-436a52ff23esm49254365ab.14 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.1.0 OK e9e14a558f8ab-436a52ff23esm49254365ab.14 

✓ Email sent successfully to krishivthakuria@gmail.com
Connecting to SMTP server for itskrishiv@gmail.com...


...

reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo 1.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.ip6.arpa\r\n'


Logging in...


reply: b'250-smtp.gmail.com at your service, [99.235.156.91]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [99.235.156.91]\nSIZE 35882577\n8BITMIME\nAUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'AUTH PLAIN AG5leHR2b3RlcnNAZ21haWwuY29tAHhycWFncXVzaXNqbXpoamQg\r\n'
reply: b'235 2.7.0 Accepted\r\n'
reply: retcode (235); Msg: b'2.7.0 Accepted'
send: 'mail FROM:<nextvoters@gmail.com> size=24118\r\n'
reply: b'250 2.1.0 OK ca18e2360f4ac-9498ffef1casm430260739f.14 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.1.0 OK ca18e2360f4ac-9498ffef1casm430260739f.14 - gsmtp'
send: 'rcpt TO:<hemitvpatel@gmail.com>\r\n'
reply: b'250 2.1.5 OK ca18e2360f4ac-9

✓ Email sent successfully to hemitvpatel@gmail.com

All emails processed.


reply: b'250 2.0.0 OK  1764477580 ca18e2360f4ac-9498ffef1casm430260739f.14 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1764477580 ca18e2360f4ac-9498ffef1casm430260739f.14 - gsmtp'
data: (250, b'2.0.0 OK  1764477580 ca18e2360f4ac-9498ffef1casm430260739f.14 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection ca18e2360f4ac-9498ffef1casm430260739f.14 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection ca18e2360f4ac-9498ffef1casm430260739f.14 - gsmtp'
